In [1]:
import numpy as np
import matplotlib.pyplot as plt

import oqupy
import oqupy.operators as op

from scipy.optimize import minimize, Bounds
from typing import List,Tuple

# --- Parameters --------------------------------------------------------------

# -- time steps --
dt = 0.05 # 0.2
num_steps = 20 # 20

# -- bath --
alpha =  0.126 #0.08
omega_cutoff = 3.04 #4
temperature =  5 * 0.1309 #1.6
pt_dkmax =60 # 40
pt_epsrel = 10**(-7) #1.0e-5

# -- initial and target state --
initial_state = op.spin_dm('x-')
target_state = op.spin_dm('x+')

# -- initial parameter guess --
x0= np.zeros(num_steps)
z0 = np.ones(num_steps) * (np.pi) / (dt*num_steps)
'''
for i in range(len(z0)):
    if i % 2 == 0:
        z0[i]=0
'''

#parameter_list=[i[0] for i in zip(x0,z0)]

parameter_list=[item for pair in zip(x0, z0) for item in pair] # this makes a list without the inner tuples

#parameter_list = list(zip(x0,z0))
#num_params = len(parameter_list[0])

In [2]:
halfparalist=[item for item in zip(x0,z0) for i in range(2)]

In [56]:
halfparalist

[(0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0, 0),
 (0, 0),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.1

In [40]:
halfparalist[2]=(0,0)
halfparalist[3]=(0,0)

[(0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0, 0),
 (0, 0),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.141592653589793),
 (0.0, 3.1

In [3]:
# --- Compute process tensors -------------------------------------------------
correlations = oqupy.PowerLawSD(alpha=alpha,
                                zeta=3,
                                cutoff=omega_cutoff,
                                cutoff_type='gaussian',
                                temperature=temperature)
bath = oqupy.Bath(oqupy.operators.sigma("z")/2, correlations)


pt_tempo_parameters = oqupy.TempoParameters(
    dt=dt,
    epsrel=pt_epsrel,
    dkmax=pt_dkmax)
process_tensor = oqupy.pt_tempo_compute(
    bath=bath,
    start_time=0.0,
    end_time=num_steps * dt,
    parameters=pt_tempo_parameters,
    progress_type='bar')

--> PT-TEMPO computation:
100.0%   20 of   20 [########################################] 00:00:00
Elapsed time: 0.4s


In [4]:
def hamiltonian(hx,hz):

    hx_sx = 0.5 * oqupy.operators.sigma('x') * hx
    hz_sz = 0.5 * oqupy.operators.sigma('z') * hz

    return hz_sz + hx_sx

parameterized_system=oqupy.ParameterizedSystem(hamiltonian)



21


In [19]:
import tensornetwork as tn

In [53]:
fsadj=oqupy.compute_gradient_and_dynamics(
        system=parameterized_system,
        initial_state=initial_state,
        target_state=target_state,
        process_tensor=process_tensor,
        parameters=halfparalist)[0][1]
firsthalfprop,secondhalfprop=parameterized_system.get_propagators(dt,halfparalist)(1)
fpt=tn.Node(firsthalfprop.T)
spt=tn.Node(secondhalfprop.T)
fsadj[0] ^ fpt[0]
fsadj[1] ^ fpt[1]
result1=fsadj @ fpt
print(result.tensor)

21
[[ 0.2499993 -4.51242786e-17j -0.20930269-1.64724789e-02j
  -0.20930269+1.64724789e-02j  0.2499993 +4.84234469e-17j]
 [-0.2499993 +4.51242786e-17j  0.20930269+1.64724789e-02j
   0.20930269-1.64724789e-02j -0.2499993 -4.84234469e-17j]
 [-0.2499993 +4.51242786e-17j  0.20930269+1.64724789e-02j
   0.20930269-1.64724789e-02j -0.2499993 -4.84234469e-17j]
 [ 0.2499993 -4.51242786e-17j -0.20930269-1.64724789e-02j
  -0.20930269+1.64724789e-02j  0.2499993 +4.84234508e-17j]]


In [54]:
fsadj=oqupy.compute_gradient_and_dynamics(
        system=parameterized_system,
        initial_state=initial_state,
        target_state=target_state,
        process_tensor=process_tensor,
        parameters=halfparalist)[0][0]
firsthalfprop,secondhalfprop=parameterized_system.get_propagators(dt,halfparalist)(0)
fpt=tn.Node(firsthalfprop.T)
spt=tn.Node(secondhalfprop.T)
fsadj[2] ^ spt[0]
fsadj[3] ^ spt[1]
result2=fsadj @ spt
print(result.tensor)

21
[[ 0.2499993 -4.51242786e-17j -0.20930269-1.64724789e-02j
  -0.20930269+1.64724789e-02j  0.2499993 +4.84234469e-17j]
 [-0.2499993 +4.51242786e-17j  0.20930269+1.64724789e-02j
   0.20930269-1.64724789e-02j -0.2499993 -4.84234469e-17j]
 [-0.2499993 +4.51242786e-17j  0.20930269+1.64724789e-02j
   0.20930269-1.64724789e-02j -0.2499993 -4.84234469e-17j]
 [ 0.2499993 -4.51242786e-17j -0.20930269-1.64724789e-02j
  -0.20930269+1.64724789e-02j  0.2499993 +4.84234508e-17j]]


In [6]:
bpl=oqupy.compute_gradient_and_dynamics(system=parameterized_system,
                                    parameters=halfparalist,
                                    initial_state=initial_state,
                                    target_state=target_state.T,
                                    process_tensor=process_tensor)

In [10]:
bpl[0][:]

[
 Edge(Dangling Edge)[0] ,
 
 Edge(Dangling Edge)[1] ,
 
 Edge(Dangling Edge)[2] ,
 
 Edge(Dangling Edge)[3] ]

In [45]:
parameterized_system.get_propagator_derivatives(dt,halfparalist)(0)[0][0]

array([[ 0.        +0.j        ,  0.00049062+0.01248715j,
         0.00049062-0.01248715j,  0.        +0.j        ],
       [ 0.00049062+0.01248715j,  0.        +0.j        ,
         0.        +0.j        , -0.00049062-0.01248715j],
       [ 0.00049062-0.01248715j,  0.        +0.j        ,
         0.        +0.j        , -0.00049062+0.01248715j],
       [ 0.        +0.j        , -0.00049062-0.01248715j,
        -0.00049062+0.01248715j,  0.        +0.j        ]])

In [48]:
parameterized_system.get_propagator_derivatives(dt,halfparalist)(1)[1][0]

array([[0.+0.j    , 0.+0.0125j, 0.-0.0125j, 0.+0.j    ],
       [0.+0.0125j, 0.+0.j    , 0.+0.j    , 0.-0.0125j],
       [0.-0.0125j, 0.+0.j    , 0.+0.j    , 0.+0.0125j],
       [0.+0.j    , 0.-0.0125j, 0.+0.0125j, 0.+0.j    ]])

In [58]:
bpl=oqupy.compute_gradient_and_dynamics(system=parameterized_system,
                                    parameters=halfparalist,
                                    initial_state=initial_state,
                                    target_state=target_state.T,
                                    process_tensor=process_tensor)[0]
props=parameterized_system.get_propagators(dt,halfparalist)
pds=parameterized_system.get_propagator_derivatives(dt,halfparalist)
thispara=0
for i in range(num_steps):
    fhprd,shprd=pds(i)
    fhprd=fhprd[thispara]
    shprd=shprd[thispara]
    fhprp,shprp=props(i)
    fpt=tn.Node(firsthalfprop.T)
    spt=tn.Node(secondhalfprop.T)
    thisadjoint=bpl[i]
    


21
